In [125]:
%load_ext jupyternotify
import numpy as np
import pandas as pd
import glob
import pickle

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [126]:
# Pickling functions used to save and load the dictionary file

def save_obj(obj, name):
    with open('dataset/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('dataset/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [321]:
# Importing UN data
data=[]
name=[]
for file_name in glob.glob('dataset/UNDP_HDI/*.csv'):
    data.append(pd.read_csv(file_name,
                             skiprows=0,
                             na_values="..",
                             na_filter=True,
                             header=1))
    name.append(file_name.split('/')[-1].split('.')[0])

# This snippet is for dealing with the format of the imported dataframes
for i in range(len(data)):
    for col in data[i].columns:
        # Drop columns containing 'Unnamed'
        if "Unnamed" in col: data[i].drop(labels=col, axis=1, inplace=True) 

In [ ]:
data_train = pd.read_csv('dataset/train.csv')
columns_train = list(data_train.columns)

In [133]:
# WARNING!------------------------------------------------------------------------
# This code snippet is not to be run if one wants to get a map
# of columns. In that case refer to the *pickled dictionary file* in the dataset
# folder. This is due to the manual corrections present later in this block, which
# are dependant on a certain sequence of columns from UN data, which in turn
# depends on the glob order.

# This code is kept only to record my steps. 

import re

# Function str_comp compares how similar two strings are
# based on matching words or numbers. It's very crude, but
# gets the job done relatively well for my purpose. 
# It returns a value from 0 to 1 (if duplicate words present could be more than 1).
# 0 means strings lstr1 and lstr2 contain completely different words
# and the larger the metric — the better similarity. 

def str_comp(lstr1, lstr2): # lstr1 and lstr2 are strings to be compared
    
    # Forming 2 lists of words from strings, 
    # ignoring all non alphabet/number symbols.
    # filter removes unnecessary blank strings in the list
    # left by re.split.
    l1 = list(filter(None, re.split(r'\W',lstr1)))
    l2 = list(filter(None, re.split(r'\W',lstr2)))
    
    # matchings DO account for duplicate words in one or both lists of strings,
    # in which case it inflates the final metric to more than 1.
    # Ideally it should be fixed, but it works fine for comparing strings. 
    matchings = 0
    for s1 in l1:
        for s2 in l2:
            if s1.lower() == s2.lower(): matchings+=1
    return matchings/max([len(l1), len(l2)])


# Making a list containing indices to match columns between datasets
# based on the str_comp function 'metric'
col_map=[]
for i, col1 in enumerate(name):
    maxim = 0
    j_m = 0
    for j, col2 in enumerate(columns_train):
        comp = str_comp(col1,col2)
        # This part checks whether this combination of list of strings col1 and col2
        # are more similar than the previous best in this loop. It also checks
        # that the second list of strings col2 has not appeared before in
        # col_map to avoid duplicates. 
        if (comp >= maxim) and (j not in [x[1] for x in col_map]): 
            maxim = comp
            j_m = j
    col_map.append((i, j_m))

# Manually fixing mistakes made by the mapping script
# NOTE! This manual corrections may change on different loads of the data
#
# Please, use the columns dict pickle file in the dataset folder 
# for final mapping. 

corrections = {
    (37, 72): (37, 8),
    (43, 78): (43, 72),
    (56, 41): (56, 42),
    (57, 8): (57, 78),
    (10, 42): (10,41),
}

# Final list with corrections:
col_map_corr = [corrections.get(x,x) for x in col_map]

# Now it's nice and dandy
for i, j in col_map_corr:
    print(i, name[i])
    print(j, columns_train[j], '\n')

0 Education Index
9 Education Index 

1 Employment to population ratio (% ages 15 and older)
34 Employment to population ratio (% ages 15 and older) 

2 Population, ages 65 and older (millions)
20 Population, ages 65 and older (millions) 

3 Mortality rate, infant (per 1,000 live births)
17 Mortality rate, infant (per 1,000 live births) 

4 Human Development Index (HDI), female
70 Intergalactic Development Index (IDI), female 

5 Estimated gross national income per capita, female (2011 PPP$)
56 Estimated gross galactic income per capita, female 

6 Share of seats in parliament (% held by women)
47 Share of seats in senate (% held by female) 

7 Labour force participation rate (% ages 15 and older), male
33 Labour force participation rate (% ages 15 and older), male 

8 Remittances, inflows (% of GDP)
62 Remittances, inflows (% of GGP) 

9 International inbound tourists (thousands)
64 Intergalactic inbound tourists (thousands) 

10 Infants lacking immunization, measles (% of one-year-ol


This code was used to save the pickled dict.  
```python
col_map_abs = {}
for i, j in col_map_corr:
    col_map_abs[name[i].strip()] = columns_train[j].strip()

save_obj(col_map_abs, 'column_dict_from_UN_to_Contest_format')
```

In [323]:
# Index rows of each dataframe in data by Country column
for i in range(len(data)):
    data[i].set_index('Country', inplace=True)

In [325]:
# Select only rows with countries and columns with years and the country list
for i,df in enumerate(data):
    data[i] = df.iloc[0:189, 1:]

In [212]:
i=37
print(name[i])
d = data[i].iloc[0:189, 1:]
d

Human Development Index (HDI)


,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,0.298,0.304,0.312,0.308,0.303,0.327,0.331,0.335,0.339,...,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
1,Albania,0.644,0.625,0.608,0.611,0.617,0.629,0.639,0.639,0.649,...,0.729,0.740,0.759,0.771,0.781,0.787,0.788,0.788,0.789,0.791
2,Algeria,0.578,0.582,0.589,0.593,0.597,0.602,0.610,0.619,0.629,...,0.720,0.730,0.738,0.737,0.746,0.749,0.751,0.755,0.758,0.759
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.830,0.828,0.827,0.849,0.846,0.853,0.850,0.854,0.852,0.857
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.508,0.510,0.525,0.537,0.547,0.557,0.565,0.570,0.576,0.574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,Venezuela (Bolivarian Republic of),0.638,0.648,0.654,0.656,0.657,0.660,0.662,0.666,0.668,...,0.752,0.753,0.764,0.767,0.772,0.770,0.763,0.752,0.735,0.726
185,Viet Nam,0.475,0.484,0.496,0.506,0.517,0.529,0.540,0.539,0.559,...,0.650,0.653,0.663,0.668,0.673,0.675,0.680,0.685,0.690,0.693
186,Yemen,0.392,0.396,0.395,0.398,0.398,0.393,0.408,0.418,0.430,...,0.503,0.499,0.511,0.501,0.506,0.504,0.493,0.477,0.463,0.463
187,Zambia,0.424,0.421,0.420,0.422,0.418,0.419,0.419,0.420,0.419,...,0.521,0.531,0.541,0.552,0.559,0.565,0.570,0.580,0.589,0.591


In [331]:
d.loc[:,'1990']

0      0.298
1      0.644
2      0.578
3        NaN
4        NaN
       ...  
184    0.638
185    0.475
186    0.392
187    0.424
188    0.498
Name: 1990, Length: 189, dtype: object

In [254]:
d[d['Country'] == 'Afghanistan']

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,0.298,0.304,0.312,0.308,0.303,0.327,0.331,0.335,0.339,...,0.447,0.464,0.465,0.479,0.485,0.488,0.49,0.491,0.493,0.496


In [266]:
d['1990']

0      0.298
1      0.644
2      0.578
3        NaN
4        NaN
       ...  
184    0.638
185    0.475
186    0.392
187    0.424
188    0.498
Name: 1990, Length: 189, dtype: object

In [246]:
cols = [int(x) for x in d.columns[1:]]

In [355]:
idx = pd.IndexSlice

undp_ind = pd.MultiIndex.from_product([cols, d['Country']], names=['year', 'country'])
undp_data = pd.DataFrame(None, index = undp_ind)

In [370]:
undp_data.loc[1990, 'Palau']

Education Index                                         NaN
Employment to population ratio (% ages 15 and older)    NaN
Name: (1990, Palau), dtype: object

In [274]:
undp_data.loc[(1990, 'Afghanistan'), 'HDI'] = d.loc[0,'1990']

In [391]:
i=0
for df, nam in zip(data, name):
    for country in df.index:
        for year in df.columns:
            undp_data.loc[(int(year), country.strip()), nam] = df.loc[country,'{}'.format(year)]

In [397]:
undp_data.loc[(2006, 'Germany')]

Education Index                                                                    0.911
Employment to population ratio (% ages 15 and older)                                 NaN
Population, ages 65 and older (millions)                                             NaN
Mortality rate, infant (per 1,000 live births)                                       NaN
Human Development Index (HDI), female                                                NaN
                                                                                   ...  
Tuberculosis incidence (per 100,000 people)                                          NaN
Mortality rate, male adult (per 1,000 people)                                        NaN
Population, total (millions)                                                         NaN
Young age (0-14) dependency ratio (per 100 people ages 15-64)                        NaN
Population with at least some secondary education, female (% ages 25 and older)      NaN
Name: (2006, Germany)

In [362]:
data[0].loc['Palau']

1990      NaN
1991      NaN
1992      NaN
1993      NaN
1994      NaN
1995      NaN
1996      NaN
1997      NaN
1998      NaN
1999      NaN
2000    0.761
2001    0.762
2002    0.763
2003    0.765
2004    0.766
2005    0.767
2006    0.773
2007    0.779
2008    0.784
2009     0.79
2010    0.795
2011    0.801
2012    0.803
2013    0.882
2014    0.863
2015    0.829
2016    0.838
2017    0.847
2018    0.845
Name: Palau, dtype: object